<a href="https://colab.research.google.com/github/butchland/fastai_nb_explorations/blob/master/fastai_scratch_with_tpu_mnist_4_experiment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
!curl https://course.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   322  100   322    0     0   1463      0 --:--:-- --:--:-- --:--:--  1463
Updating fastai...
Done.


In [3]:
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4139  100  4139    0     0  49867      0 --:--:-- --:--:-- --:--:-- 49867
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
     |████████████████████████████████| 61kB 2.7MB/s 
Uninstalling torch-1.5.1+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
\
Operation completed over 1 objects/83.4 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200325-cp36-cp36m-

In [4]:
!pip freeze | grep torchvision
!pip freeze | grep torch-xla

torchvision==0.6.0a0+3c254fb
torch-xla==1.6+e788e5b


In [5]:
!pip install fastcore --upgrade

In [6]:
!pip install fastai2 --upgrade

     |████████████████████████████████| 194kB 3.4MB/s 


In [7]:
pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.6/dist-packages (1.0.61)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
%cd /content/drive/My\ Drive/course-v4/
!pwd

/content/drive/My Drive/course-v4
/content/drive/My Drive/course-v4


In [9]:
!pip install -r requirements.txt

     |████████████████████████████████| 51kB 1.8MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 1.1MB 10.7MB/s 
     |████████████████████████████████| 92kB 8.4MB/s 
     |████████████████████████████████| 40kB 4.6MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 2.7MB 32.2MB/s 


In [10]:
%cd nbs

/content/drive/My Drive/course-v4/nbs


In [11]:
!pwd

/content/drive/My Drive/course-v4/nbs


### Start of import libraries

In [12]:
from fastai2.vision.all import *

In [13]:
from utils import *

In [14]:
path = untar_data(URLs.MNIST_SAMPLE)

In [15]:
Path.BASE_PATH = path

In [16]:
path.ls()

(#3) [Path('valid'),Path('labels.csv'),Path('train')]

### Import torch xla libraries

In [17]:
import torch

In [18]:
import torch_xla

In [19]:
import torch_xla.core.xla_model as xm

In [20]:
OptimWrapper?


In [29]:

class WrapperOpt:
    def __init__(self, f):
        self.f = f 
    def __call__(self, *args, **kwargs):
        opt = self.f(*args, **kwargs)
        optim_wrapper = OptimWrapper(opt)
        def my_step():
            xm.optimizer_step(opt,barrier=True)
        optim_wrapper.step = my_step
        return optim_wrapper
    
def wrap_xla_optim(opt):
    w = WrapperOpt(opt)
    return w

### Get TPU Device

In [23]:
tpu_dev = xm.xla_device()

In [24]:
tpu_dev

device(type='xla', index=1)

In [25]:
datablock = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW),CategoryBlock),
    get_items=get_image_files,
    splitter=GrandparentSplitter(),
    get_y=parent_label,
    item_tfms=Resize(28),
    batch_tfms=[])


In [26]:
dls = datablock.dataloaders(path,device=tpu_dev)

In [30]:
adam_xla_opt = wrap_xla_optim(Adam)

In [18]:
sgd_xla_opt = wrap_xla_optim(SGD)

In [45]:
learner = cnn_learner(dls, resnet18, metrics=accuracy, 
                      loss_func=F.cross_entropy, opt_func=adam_xla_opt)

In [43]:
from fastai2.callback.tensorboard import *

In [47]:
learner.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.041712,0.030968,0.989205,00:25
1,0.025518,0.025715,0.992640,00:22
2,0.021731,0.015921,0.996075,00:22


In [37]:
!pip freeze | grep tensorboard

tensorboard==2.2.2
tensorboard-plugin-wit==1.6.0.post3
tensorboardcolab==0.0.22
